In [11]:
import joblib
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [12]:
# import csv for red
# https://stackoverflow.com/questions/24606330/how-to-read-a-file-with-a-semi-colon-separator-in-pandas

red_wines = pd.read_csv("winequality-red.csv", sep=";")
red_wines

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [13]:
red_wines['target'] = np.where(red_wines['quality']<=5,0,1)

In [14]:
# Assign X and y
X = red_wines.drop(["quality", "target"], axis=1)
y= red_wines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [15]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [16]:
from sklearn.preprocessing import StandardScaler

In [18]:
X_scaler= StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators': [5, 50, 250, 500],
    'max_depth': [1, 3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 1, 10, 100]
}

cv = GridSearchCV(gb, parameters, cv=5)
cv.fit(X_train_scaled, y_train)

print_results(cv)

BEST PARAMS: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 500}

0.528 (+/-0.003) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 5}
0.714 (+/-0.072) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}
0.732 (+/-0.092) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 250}
0.752 (+/-0.088) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500}
0.528 (+/-0.003) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 5}
0.728 (+/-0.072) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
0.765 (+/-0.092) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 250}
0.767 (+/-0.085) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
0.528 (+/-0.003) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 5}
0.755 (+/-0.068) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
0.779 (+/-0.077) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 250}
0.784 (+/-0.056) for {'learning_r

In [20]:
cv.best_estimator_

GradientBoostingClassifier(max_depth=7, n_estimators=500)